## **Imports**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage import io, transform
import torch
from tqdm import tqdm
import os
import time
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import copy
from skimage import io, transform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
dataset_train = pd.read_csv("data/train.csv")
dataset_validation = pd.read_csv("data/val.csv")


In [3]:
device

device(type='cpu')

In [4]:
model_101 = models.resnet101(pretrained=True)

In [5]:
model_101

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## **Defining model**

In [6]:
for param in model_101.parameters():
  param.requires_grad = False
import torch.nn as nn

n_inputs = model_101.fc.in_features
last_layer = nn.Linear(n_inputs, 1)
model_101.fc = last_layer
model_101.to(device)
print(model_101.fc.out_features)

1


# **LoadModel**

In [7]:
weights_dict = torch.load("models/bestmodel101.pt")

In [8]:
model_101.load_state_dict(weights_dict["model_state_dict"])

<All keys matched successfully>

In [9]:
model_101 = model_101.to(device)

In [10]:
model_101.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
# defining hook
feature_blobs = []


def hook_feature(module, input, output):
    feature_blobs.append(output.cpu().data.numpy())

In [12]:
finalconv_name = "layer4"

In [13]:
model_101._modules.get(finalconv_name).register_forward_hook(hook_feature)

In [14]:
# feature_blobs

In [15]:
params = list(model_101.parameters())

In [16]:
weight_softmax = np.squeeze(params[-3].cpu().data.numpy())

In [17]:
params[-3].shape

torch.Size([2048])

In [18]:
weight_softmax

array([ 0.02827137,  0.00630928,  0.00824687, ...,  0.00714007,
       -0.00359139, -0.00576739], dtype=float32)

In [19]:
weight_softmax.shape

(2048,)

## **Loading dataset**

In [20]:
class AsetheticsDataset(Dataset):
    '''asethitics dataset'''

    def __init__(self, dataframe, root_dir, transform=None):
        """
            Args:
                csv_file (string): Path to the csv file with annotations.
                root_dir (string): Directory with all the images.
                transform (callable, optional): Optional transform to be applied
                    on a sample.
        """
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = io.imread(image_name)
        mem_val = self.data.iloc[idx, 1]
        return_sample = {
            'image': image,
            'memorability_score': mem_val
        }
        if self.transform:
            return_sample = self.transform(return_sample)

        return return_sample


class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image, mem_val = sample['image'], sample["memorability_score"]

        h, w = image.shape[:2]

        img = transform.resize(image, (self.output_size, self.output_size, 3))
        return {'image': img, 'memorability_score': mem_val}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, mem_val = sample['image'], sample['memorability_score']
        image = image.transpose((2, 0, 1))

        return {'image': torch.from_numpy(image),
                'memorability_score': mem_val}


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        image, mem_val = sample["image"], sample["memorability_score"]
        normalized = (image - self.mean) / self.std
        return {
            "image": normalized,
            "memorability_score": mem_val
        }

In [21]:
from PIL import Image
import io

In [22]:
dataset = pd.read_csv("data/dataset.csv")

In [23]:
for index, row in dataset.iterrows():

    normalize = transforms.Normalize((0.5,), (0.5,))
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize
    ])

    img_pil = Image.open('images/' + row["image"])

    img_tensor = preprocess(img_pil)
    img_tensor = img_tensor.to(device)
    img_tensor = img_tensor.unsqueeze(0)
    mem_score = model_101(img_tensor)

    size_upsample = (224, 224)
    bz, nc, h, w = feature_blobs[-1].shape
    cam = np.dot(weight_softmax.reshape(1, -1),
                 feature_blobs[-1].reshape((nc, h*w)))
    cam = cam.reshape(h, w)

    cam = cam - np.min(cam)
    cam_img = cam / np.max(cam)
    cam_img = np.uint8(255 * cam_img)

    up_sampled_image = cv2.resize(cam_img, size_upsample)

    img = cv2.imread('images/' + row["image"])
    height, width, _ = img.shape
    heatmap = cv2.applyColorMap(cv2.resize(
        up_sampled_image, (width, height)), cv2.COLORMAP_HOT)
    result = heatmap * 0.3 + img * 0.5
    cv2.imwrite('map/' + row["image"], result)